# Ejercicio 1

In [1]:
import pandas as pd
import numpy as np

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Conv1D, Flatten, Dropout, BatchNormalization, MaxPooling2D, LSTM, GRU
from keras.optimizers import Adam

2022-09-21 16:49:53.489440: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-21 16:49:53.722560: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-21 16:49:53.722582: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-21 16:49:53.813594: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-21 16:49:54.640695: W tensorflow/stream_executor/platform/defaul

## 1.1

- Tenemos un problema de clasificación de objetos donde tenemos 5000 imágenes de 28x28 en color. Para cada objeto tenemos una etiqueta numérica con valores de [0-19] (20 clases de objetos distintas). Propon un modelo teniendo en cuenta lo que has aprendido en clase.

__COMENTARIOS__ 

* Implemento un modelo muy simple. De esta manera me servirá para ver como mejora según le vaya añadiendo cosas en los siguiente ejercicios.

* Probablemente sería mejor añadir una segunda capa Conv2D

* Añado capa de maxpool para disminuir el número de parámetros ya que no tengo muchos datos. Aunque, en realidad, tenemos muchos datos 26 * 26 * 3 * 5000 = 10.140.000

* La primera capa Dense sirve para seleccionar los patrones más relevantes de todos los que selecciona la capa Conv2d y la segunda para calcular las probabilidades de cada una de las 20 clases

In [15]:
model_1 = Sequential()
model_1.add(Conv2D(32, 3, input_shape=(28, 28, 3), activation='relu'))
model_1.add(MaxPooling2D((2,2)))
model_1.add(Flatten())
model_1.add(Dense(33, activation='relu'))
model_1.add(Dense(20, activation='softmax'))

model_1.summary()

model_1.compile(optimizer='adam', loss='categorical_cross_entropy')

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 26, 26, 32)        896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 5408)              0         
                                                                 
 dense_8 (Dense)             (None, 33)                178497    
                                                                 
 dense_9 (Dense)             (None, 20)                680       
                                                                 
Total params: 180,073
Trainable params: 180,073
Non-trainable params: 0
_______________________________________________

## 1.2

- Tenemos 500 datos de series temporales de un activo financiero, cada dato tiene una longitud de 7. Las salidas son las predicciones para el instante temporal siguiente. Escribe un trozo de código que sirva para definir un modelo en Keras que tenga al menos dos capas LSTM, una capa GRU y una densa.

__COMENTARIOS:__ 

* El modelo tiene solo 1229 parámetros ya que tenemos muy pocos datos

* Es fundamental el orden de las distintas capas:

    * Las dos primeras capas son many to many y la tercera es many to one

    * Las capas LSTM tienen memoria corta y larga por lo que funcionan bien en configuraciones many to many

    * Las capas GRU solo tienen memoria a largo y encajan más en las configuraciones many to one
    
    * Por esto, ponemos primero las capas LSTM y luego la GRU

In [13]:
n_datos = 500
datos_length = 7

model_2 = Sequential()

model_2.add(LSTM(8, input_shape=(n_datos, datos_length), return_sequences=True))
model_2.add(LSTM(8, return_sequences=True))
model_2.add(GRU(4))
model_2.add(Dense(1))

model_2.compile(optimizer='adam',loss='mae')

model_2.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 500, 8)            512       
                                                                 
 lstm_7 (LSTM)               (None, 500, 8)            544       
                                                                 
 gru_1 (GRU)                 (None, 4)                 168       
                                                                 
 dense_5 (Dense)             (None, 1)                 5         
                                                                 
Total params: 1,229
Trainable params: 1,229
Non-trainable params: 0
_________________________________________________________________


## 1.3

- Tenemos 50000 datos de series temporales de 10 activos financieros de longitud 13, i.e. (50000, 13, 10). La salida son las 4 posibles fases del investment clock (reflation, recovery, overheat, y stagflation) para el instante temporal siguiente. Escribe un trozo de código que sirva para definir un modelo en Keras que combine al menos una capa LSTM, dos convolucionales y una densa.

__Comentarios:__

* Pongo primero las capas convolucionales ya que busca patrones importantes y no pierde la noción temporal y luego ya uso las LSTM

In [9]:
model_3 = Sequential()

model_3.add(Conv1D(64, 3, input_shape=(13, 10)))
model_3.add(Conv1D(128, 3))
model_3.add(LSTM(32))
model_3.add(Dense(4, activation="softmax"))

model_3.compile(optimizer= Adam(),
              loss='categorical_crossentropy')

model_3.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 11, 64)            1984      
                                                                 
 conv1d_3 (Conv1D)           (None, 9, 128)            24704     
                                                                 
 lstm_2 (LSTM)               (None, 32)                20608     
                                                                 
 dense_3 (Dense)             (None, 4)                 132       
                                                                 
Total params: 47,428
Trainable params: 47,428
Non-trainable params: 0
_________________________________________________________________
